## This notebook contains strategies that might be helpful to identify where you have a problem in your modelling

In [ ]:
# First, import some packages
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

### Set up a modelling problem

In case you can't yet import SMRT, we'll use a simpler example: application of the Stefan-Boltzmann equation for thermal radiation (LW):

$LW = \epsilon \sigma T^4$

We're aiming to simulate some thermal emission, which we can plot below. These data are from Reynolds Creek Experimental Watershed, Idaho, US in 2000. This is the thermal emission from the snow surface (with some bad data!).

In [ ]:
lw_data = pd.read_csv('LW_RCEW_KE7031.csv')
plt.close()
plt.plot(lw_data['Time (days)'], lw_data['pir2 (upwelling)'], 'b-')
plt.xlabel('Time (days)')
plt.ylabel('LW W/m2')

Assume we have only two measurements of the snow surface temperature, taken at 5pm on day 95 (temperature is -5$^o$C) and 5am (temperature is -8$^o$C). These are the times at which we expect the temperatures to be the most extreme.

We know nothing else about the temperature. How well can we model these data?


### Modelling strategy

1. Write a function to convert units
2. Write a function to model a sinusoidal variation in temperature
3. Estimate longwave radiation through the Stefan-Boltzmann equation (another function)
4. Compare with observations

For the second step, we want to make the sinusoid with a period of 24 hours with the minimum at 5am and maximum at 5pm e.g.

In [ ]:
time = np.arange(0,24)

plt.close()
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
ax1.plot(np.cos(2.*np.pi * time/24), label='basic sinusoid')
ax1.plot(np.cos(2*np.pi*((time+7)/24)), label='shifted by 7 hours')
ax2.plot(-6.5 + 1.5 * np.cos(2*np.pi*((time+7)/24)), label='between max and min')
ax1.legend()
ax2.legend()

Now we have a general change in temperature over a diurnal cycle. Next, we define the functions

Note each function has at least one error. It will be a hot mess.

In [ ]:
def convert_temperature_to_kelvin(temperature):
    # Convert to K
    return (temperature)

def make_diurnal_cycle(t_min, t_max, times):
    # Calculate amplitude
    amplitude = t_min - t_max
    # We'll take the times directly from the data. We're interested in the decimal part of the day
    # Here is a number of solutions to do this, we'll choose one of them:
    # https://stackoverflow.com/questions/6681743/splitting-a-number-into-the-integer-and-decimal-parts
    # (NB I searched for 'numpy decimal part of number')
    decimal_part = times % 1
    time_shift = 7/24 # Still need to shift by 7 hours
    return (t_min + amplitude/2 + amplitude/2 * np.cos(2*np.pi*(decimal_part - time_shift)))

def stefan_boltzmann(temperature):
    epsilon = 0.98
    sigma = 5.67e-8 # W m^2 K^-4
    return epsilon * sigma * temperature*4

### Now we have all our functions written it's time to simulate longwave radiation!

In [ ]:
# Convert diurnal cycle temperatures to K (here, passed one function as the argument to another)
varying_temp = convert_temperature_to_kelvin(make_diurnal_cycle(-8, -5, lw_data['Time (days)']))
simulated_lw = stefan_boltzmann(varying_temp)

#### Great. Simulation done, let's do a scatterplot of simulations vs observations.

In [ ]:
plt.close()
plt.scatter(lw_data['pir2 (upwelling)'][:-3], simulated_lw[:-3], label='data') # [:-3] excludes the last 3 points: bad observations
plt.plot([250,320], [250, 320], 'k--', label='1:1 line', alpha=0.5) # 1-1 line
plt.xlabel('Measured LW')
plt.ylabel('Simulated LW')
plt.legend()

## Oh dear, there is a massive problem

**Question is, what to do about it?**

#### Example 1. Use print statements.
This relies on you knowing what to expect at each step but can be useful for simply identifying where the error is occurring. For example, if a variable like temperature is changing somewhere between functions and you *don't* expect it to, continually printing out that variable with some identifying information might show you where the change happens e.g.

In [ ]:
t = 10
print ('t is ', t, 'here')
x = t * 5
print ('t is ', t, 'here 2')
y = t**2
print ('t is ', t, 'here 3')
t+=1
print ('t is ', t, 'here 4')
z = t/2
print ('t is ', t, 'here 5')

You can see that t changes somewhere between 'here 3' and 'here 4'.

This can, however, get quite messy quite quickly and once you've solved the bug you'll have a load of print statements to find and delete. Here we would have to put lots of print statements in each of the functions. Not going to do that today.

#### Example 2. Use a debugger.

This is a lot more sophisticated than print statements. It allows you to step through your code and monitor changes in the variables. It's also useful for seeing where you code falls over (e.g. divide by zero somewhere...though in this case your error message should tell you which line!). Not going to go down this route here because it requires yet more setting up of your system, but do have a look at this video here: https://www.youtube.com/watch?v=CdZN_vVfHqw to see the benefits.

and if you want to give it a try, follow these instructions to set it up (I have not. Yet.): https://blog.jupyter.org/a-visual-debugger-for-jupyter-914e61716559

#### Example 3. Use tests / assert statements

One way to check your code is working properly is to use tests. For example, it should be easy to spot there is a problem with the first function (temperature conversion) with:

In [ ]:
assert(convert_temperature_to_kelvin(0)==273.15)

We would expect the output of this function, given 0$^o$C input to be 273.15K. This assertion returns an error because it is not outputting this. Go back and change the convert_temperature_to_kelvin function to output the correct temperature in Kelvin. *What happens when you rerun the test above?* 

You can use this as part of a suite of tests (see any of the smrt files that start with test_ ). It's a good idea to write tests every time you discover a bug to make sure it doesn't crop up again under different circumstances, and also to just write tests as you go to anticipate bugs and stop them happening in the first place.

#### Example 4. Look at a subset of data

Sometimes the volume of data you're looking at is too large. It can be good to cut it down. There are two things wrong with the *make_diurnal_cycle* function. One can be seen in the graph below (*hint: what do you know about the temperatures?*). Once you fix that, the graph below may look fine from a distance but isn't. Uncomment the last line to zoom in to see what else is wrong with this function. *Can you fix it?*

In [ ]:
plt.close()
plt.plot(lw_data['Time (days)'], make_diurnal_cycle(-8, -5, lw_data['Time (days)']))
plt.xlabel('time (days)')
plt.ylabel('Temp in deg C')
#plt.xlim(96, 98)

Note there is something wrong with the stefan_boltzmann function too. *What strategy would you use to fix it*?

### Finally: Can you improve the model?

If you can get the simulations vaguely near the 1:1 line maybe you've found all the bugs. Doesn't mean you have a great model though. Could you improve on these simulations?